In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
from translate import Translate
from spellcheck import Spellcheck
from loader import Loader

In [ ]:
cols = ["pap-simple", "nl-simple"]
to_language = "en-simple"
hny_pap_nl_raw = pd.read_csv("../data/hny/pap-nl.csv", na_filter=False, usecols=cols)
custom_pap_nl_raw = pd.read_csv("../data/custom/pap-nl.csv", na_filter=False, usecols=cols)
stpark_pap_nl_raw = pd.read_csv("../data/stparkpap/pap-nl(extracted_words).csv", na_filter=False, usecols=cols)
rblx_pap_nl_raw = pd.read_csv("../data/rblx/pap-nl.csv", na_filter=False, usecols=cols)

pap_nl_dfs = pd.concat([custom_pap_nl_raw, hny_pap_nl_raw, stpark_pap_nl_raw, rblx_pap_nl_raw], ignore_index=True)        
len(pap_nl_dfs)

In [ ]:
pap_nl_dfs_no_dupe = pap_nl_dfs.drop_duplicates(subset=cols, keep="first", ignore_index=True)
len(pap_nl_dfs_no_dupe)

In [ ]:
pap_nl_dfs_no_dupe[cols[1:]].to_excel("../data/custom/enhanced/pap-nl-source.xlsx", index=False)
pap_nl_dfs_no_dupe[cols[:]].to_csv("../data/custom/enhanced/pap-nl-source.csv")

In [ ]:
pap_en_from_pap_nl = pd.read_excel("../data/custom/enhanced/pap-en-from-pap-nl.xlsx")
pap_nl_enhanced = pd.concat([pap_nl_dfs_no_dupe[["pap-simple"]], pap_en_from_pap_nl], axis=1)
# pap_en_from_pap_nl
# pap_nl_dfs_no_dupe
pap_nl_enhanced[cols[1]] = pap_nl_enhanced[cols[1]].str.strip()
pap_nl_enhanced = pap_nl_enhanced.rename(columns={cols[1]:to_language})
pap_nl_enhanced.to_csv("../data/custom/enhanced/pap-en-from-pap-nl.csv")

In [ ]:
#Ignore extra words that already exist
lan = "PAP-EN"
load = Loader()
dictionary, dictionary_corpus = load.loadDictionary(lan)
spell = Spellcheck(spellchecker_corpus=dictionary_corpus, lan=lan, load=True)

new_translations_filter = pap_nl_enhanced.apply(lambda row: not spell.trie.find(row[cols[0]].lower()), axis=1)
new_translations = pap_nl_enhanced[new_translations_filter]
len(new_translations)

In [ ]:
new_translations.to_csv("../data/custom/enhanced/pap-en.csv")